In [101]:
import numpy as np
import pandas as pd
import os
import re
from tqdm import tqdm
import requests
from io import StringIO
from os import listdir
from os.path import isfile, join
import datetime as dt
from datetime import datetime, timedelta

In [2]:
def listdir_nohidden(path):
    for f in os.listdir(path):
        if not f.startswith('.'):
            yield f



In [3]:
FILE_PATH = "/Users/evanlipton/Dropbox (YSE)/Solar_ElectricData/Data"

In [4]:
ISOS = [file for file in os.listdir(FILE_PATH) if file[0]!='.']
print(ISOS)

['SPP', 'NYISO', 'CAISO', 'PJM', 'MISO', 'ERCOT', 'ISONE']


In [5]:
for ISO in ISOS:
    print(ISO)

SPP
NYISO
CAISO
PJM
MISO
ERCOT
ISONE


In [6]:
ISO_files = {}
ISO_data = {}
ISO_final = {}

#load in
for ISO in ISOS:
    ISO_files[ISO] = listdir_nohidden(f"{FILE_PATH}/{ISO}/old")
    ISO_data[ISO] = {}
    for file in tqdm(ISO_files[ISO]):
        compiled = re.compile(re.escape(f"{ISO}_"), re.IGNORECASE)

        ISO_data[ISO][compiled.sub("", file).replace(".csv","")] = pd.read_csv(f"{FILE_PATH}/{ISO}/old/{file}")

13it [00:01, 11.01it/s]
14it [00:02,  5.33it/s]
13it [00:01,  8.97it/s]
16it [00:02,  6.79it/s]
16it [00:00, 18.44it/s]
13it [00:01,  9.26it/s]
19it [00:01, 12.81it/s]


In [7]:
#add merge datetime
titles = {}
for key in ISO_data:
    print(key)
    titles[key]= []
    for k in ISO_data[key]:
        try:
            ISO_data[key][k]["time_tomerge"] = pd.to_datetime(ISO_data[key][k].iloc[:,0], utc=True)
        except:
            ISO_data[key][k] = ISO_data[key][k][ISO_data[key][k].index % 24 != 0]
            ISO_data[key][k]["time_tomerge"] = pd.to_datetime(ISO_data[key][k].iloc[:,0], utc=True)
            print(type(ISO_data[key][k]["time_tomerge"]))

SPP
NYISO
CAISO
PJM
MISO
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
ERCOT
ISONE


In [8]:
ISO_final = {}
def clean_final(ISO: str, df: str):
    ISO_data[ISO][df].columns = ISO_data[ISO][df].columns.str.lower()
    no_date = [col for col in ISO_data[ISO][df].columns if "date" not in col]
    ISO_data[ISO][df] = ISO_data[ISO][df][no_date]
    try:
        ISO_data[ISO][df] = ISO_data[ISO][df].drop("unnamed: 0")
        ISO_data[ISO][df] = ISO_data[ISO][df].drop("hour")
    except:
        pass
    ISO_data[ISO][df] = ISO_data[ISO][df].sort_values(by="time_tomerge")
    ISO_data[ISO][df] = ISO_data[ISO][df].set_index("time_tomerge")
    ISO_final[ISO] = ISO_data[ISO][df]
    for column in ISO_final[ISO].columns:
        if str(ISO).lower() in column:
            ISO_final[ISO] = ISO_final[ISO].rename(columns={column: column.replace(f"{str(ISO).lower()}_","")})
    print(ISO_final[ISO])

In [9]:
clean_final("ISONE", "all_data")
clean_final("ERCOT", "final")
clean_final("MISO", "final")
clean_final("PJM", "final")
clean_final("CAISO", "all_data")
clean_final("NYISO", "all_data")
clean_final("SPP", "final")



                           congestion_da  congestion_rt  energy_da  energy_rt  \
time_tomerge                                                                    
2014-01-01 05:00:00+00:00            0.0        0.00000      62.04     117.43   
2014-01-01 06:00:00+00:00            0.0        0.00000      54.65     148.38   
2014-01-01 07:00:00+00:00            0.0        0.00000      38.40      97.06   
2014-01-01 08:00:00+00:00            0.0        0.00000      38.02      38.54   
2014-01-01 09:00:00+00:00            0.0        0.00000      37.48      79.01   
...                                  ...            ...        ...        ...   
2019-01-01 00:00:00+00:00            0.0       -0.01875      69.19      24.96   
2019-01-01 01:00:00+00:00            0.0       -0.01125      68.83      21.33   
2019-01-01 02:00:00+00:00            0.0        0.00000      59.21      22.37   
2019-01-01 03:00:00+00:00            0.0        0.00000      53.27      24.91   
2019-01-01 04:00:00+00:00   

In [214]:
ISO_done = ["CAISO"]
FILE_PATH = "/Users/evanlipton/Dropbox (YSE)/Solar_ElectricData/Data"
new_data = {}
for ISO in ISO_done:
    sub_folder = f"{FILE_PATH}/{ISO}/new/combined"
    files = [f for f in listdir(sub_folder) if isfile(join(sub_folder, f)) and f[0]!="."]
    new_data[ISO] = {}
    print(ISO)
    for file in tqdm(files):
        new_data[ISO][file.replace(".csv", "")] = pd.read_csv(f"{sub_folder}/{file}")

CAISO


 44%|████▍     | 4/9 [01:11<00:57, 11.59s/it]/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3442: DtypeWarning: Columns (1,2,3,5,6,7,8,9,10,11,12,18,19,20,21,26) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
100%|██████████| 9/9 [02:20<00:00, 15.66s/it]


In [595]:
def create_date(entry: tuple):
    return pd.to_datetime(pd.to_datetime(entry[0]) + timedelta(hours = int(entry[1])))

def add_caiso_ancillary(sample_data: pd.DataFrame, ANC_TYPE: str, ANC_REGION: str, RESULT_TYPE: str):
    subset = sample_data[(sample_data["ANC_TYPE"] == ANC_TYPE[13:].upper()) & (sample_data["ANC_REGION"] == ANC_REGION) & (sample_data["RESULT_TYPE"] == RESULT_TYPE)]
    subset = subset[subset["OPR_HR"]<25]
    subset = pd.DataFrame(subset.groupby(['OPR_DT', "OPR_HR"])["MW"].mean())
    subset["DATE_HR"] = subset.index.to_numpy()
    subset["time_tomerge"] = subset["DATE_HR"].apply(create_date)
    #subset["time_tomerge"] = subset["time_tomerge"].map(lambda x: x.tz_convert('US/Pacific'))
    subset = subset.drop(columns = ["DATE_HR"])
    subset = subset.rename(columns={"MW": ANC_TYPE})
    subset = subset.drop_duplicates()
    print(len(np.unique(subset["time_tomerge"])))
    print(len(subset["time_tomerge"]))

    return subset

In [596]:
ancillaries = [col for col in ISO_final["CAISO"].columns if "ancillary" in col]

['ancillary_da_nr',
 'ancillary_da_rd',
 'ancillary_da_rmd',
 'ancillary_da_rmu',
 'ancillary_da_ru',
 'ancillary_da_sr',
 'ancillary_rt_nr',
 'ancillary_rt_rd',
 'ancillary_rt_rmd',
 'ancillary_rt_rmu',
 'ancillary_rt_ru',
 'ancillary_rt_sr']

In [597]:
new_data["CAISO"]["total"] = pd.DataFrame({"time_tomerge": pd.date_range(start="1/1/2019", end="1/1/2023", freq="H")})
print(len(new_data["CAISO"]["total"] ))
for anc in tqdm(ancillaries):
    if anc[10:12] == "da":
        new_data["CAISO"]["total"] = new_data["CAISO"]["total"].merge(add_caiso_ancillary(new_data["CAISO"]["ancillary_res_da"], anc, "AS_CAISO_EXP", "AS_COST"), on = "time_tomerge", how = "left")
    else:
        new_data["CAISO"]["total"] = new_data["CAISO"]["total"].merge(add_caiso_ancillary(new_data["CAISO"]["ancillary_res_rt"], anc, "AS_CAISO_EXP", "AS_COST"), on = "time_tomerge", how = "left")
#print(sample)

35065


  8%|▊         | 1/12 [00:10<01:57, 10.67s/it]

35060
35060


 17%|█▋        | 2/12 [00:18<01:29,  8.97s/it]

35060
35060


...

34316
34316


 92%|█████████▏| 11/12 [01:26<00:07,  7.22s/it]

34316
34316


100%|██████████| 12/12 [01:36<00:00,  8.02s/it]

34316
34316


In [598]:
ISO_final["CAISO"].columns

Index(['ancillary_da_nr', 'ancillary_da_rd', 'ancillary_da_rmd',
       'ancillary_da_rmu', 'ancillary_da_ru', 'ancillary_da_sr',
       'ancillary_rt_nr', 'ancillary_rt_rd', 'ancillary_rt_rmd',
       'ancillary_rt_rmu', 'ancillary_rt_ru', 'ancillary_rt_sr',
       'congestion_da', 'energy_da', 'unnamed: 0', 'gas_price', 'hour',
       'gen_geothermal', 'gen_biomass', 'gen_biogas', 'gen_small_hydro',
       'gen_wind', 'gen_solar_pv', 'gen_solar_thermal', 'gen_renewables',
       'gen_nuclear', 'gen_thermal', 'gen_imports', 'gen_hydro', 'load_2da',
       'load_7da', 'load_rt', 'load_da', 'load_rt.1', 'load_rtpd',
       'losses_da'],
      dtype='object')

In [602]:
def add_caiso_energy(sample_data: pd.DataFrame, LMP_TYPE: str, col_name: str):
    subset = sample_data[(sample_data["LMP_TYPE"] == LMP_TYPE)]
    subset = subset[subset["OPR_HR"]<25]

    try:
        subset = pd.DataFrame(subset.groupby(['OPR_DT', "OPR_HR"])["MW"].mean())
    except:
        subset = pd.DataFrame(subset.groupby(['OPR_DT', "OPR_HR"])["VALUE"].mean())

    subset["DATE_HR"] = subset.index.to_numpy()
    subset["time_tomerge"] = subset["DATE_HR"].apply(create_date)
    #subset["time_tomerge"] = subset["time_tomerge"].map(lambda x: x.tz_convert('US/Pacific'))

    subset = subset.drop(columns = ["DATE_HR"])
    subset = subset.reset_index(drop=True)


    subset = subset.rename(columns={"MW": col_name})
    subset = subset.rename(columns={"VALUE": col_name})
    print(len(subset))
    return subset

In [603]:
new_data["CAISO"]["total"] = new_data["CAISO"]["total"].merge(add_caiso_energy(new_data["CAISO"]["energy_da"], "MCC", "congestion_da"), on = "time_tomerge", how="left")
new_data["CAISO"]["total"] = new_data["CAISO"]["total"].merge(add_caiso_energy(new_data["CAISO"]["energy_da"], "MCE", "energy_da"), on = "time_tomerge", how="left")
new_data["CAISO"]["total"] = new_data["CAISO"]["total"].merge(add_caiso_energy(new_data["CAISO"]["energy_da"], "MCL", "losses_da"), on = "time_tomerge",how="left")

print(new_data["CAISO"]["total"])

27957
27957
27957
             time_tomerge  ancillary_da_nr  ancillary_da_rd  ancillary_da_rmd  \
0     2019-01-01 00:00:00              NaN              NaN               NaN   
1     2019-01-01 01:00:00          73.6890         4587.144               0.0   
2     2019-01-01 02:00:00          70.5892         1500.000               0.0   
3     2019-01-01 03:00:00          71.0776         1500.000               0.0   
4     2019-01-01 04:00:00          73.1093         1500.000               0.0   
...                   ...              ...              ...               ...   
35060 2022-12-31 20:00:00         101.5209         5700.000               0.0   
35061 2022-12-31 21:00:00          90.5520         6612.000               0.0   
35062 2022-12-31 22:00:00          87.7104         8850.000               0.0   
35063 2022-12-31 23:00:00          84.4800         5670.000               0.0   
35064 2023-01-01 00:00:00          80.8572         7650.000               0.0   

       an

In [605]:
def add_caiso_gas(sample_data: pd.DataFrame, col_name: str):
    subset = sample_data
    subset = subset[subset["OPR_HR"]<25]

    subset = pd.DataFrame(subset.groupby(['OPR_DT', "OPR_HR"])["PRC"].mean())
    subset["DATE_HR"] = subset.index.to_numpy()
    subset["time_tomerge"] = subset["DATE_HR"].apply(create_date)
    subset = subset.drop(columns = ["DATE_HR"])
    subset = subset.reset_index(drop=True)
    subset = subset.rename(columns={"PRC": col_name})
    print(len(subset))
    return subset

In [606]:
new_data["CAISO"]["total"] = new_data["CAISO"]["total"].merge(add_caiso_gas(new_data["CAISO"]["gas_price"], "gas_price"), on = "time_tomerge", how="left")
new_data["CAISO"]["total"]

35036


,time_tomerge,ancillary_da_nr,ancillary_da_rd,ancillary_da_rmd,ancillary_da_rmu,ancillary_da_ru,ancillary_da_sr,ancillary_rt_nr,ancillary_rt_rd,ancillary_rt_rmd,ancillary_rt_rmu,ancillary_rt_ru,ancillary_rt_sr,congestion_da,energy_da,losses_da,gas_price
0,2019-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-01-01 01:00:00,73.6890,4587.144,0.0,0.0,2451.00000,669.90000,0.0,0.734587,0.0,0.0,0.538445,0.0,NaN,NaN,NaN,5.098778
2,2019-01-01 02:00:00,70.5892,1500.000,0.0,0.0,1797.00000,641.72000,0.0,0.202735,0.0,0.0,0.165420,0.0,NaN,NaN,NaN,5.098778
3,2019-01-01 03:00:00,71.0776,1500.000,0.0,0.0,1512.00000,161.54000,0.0,0.451492,0.0,0.0,0.215452,0.0,NaN,NaN,NaN,5.098778
4,2019-01-01 04:00:00,73.1093,1500.000,0.0,0.0,1512.00000,166.15750,0.0,0.437500,0.0,0.0,0.192995,0.0,NaN,NaN,NaN,5.098778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35060,2022-12-31 20:00:00,101.5209,5700.000,0.0,0.0,2483.99900,2533.73540,0.0,106.494525,0.0,0.0,0.000000,0.0,-0.030910,140.78702,-3.402353,16.604248
35061,2022-12-31 21:00:00,90.5520,6612.000,0.0,0.0,1892.67000,1916.61000,0.0,214.631892,0.0,0.0,0.000000,0.0,-0.004903,135.76392,-3.267383,16.604248
35062,2022-12-31 22:00:00,87.7104,8850.000,0.0,0.0,4625.81000,3186.47000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,128.26624,-3.031357,16.604248
35063,2022-12-31 23:00:00,84.4800,5670.000,0.0,0.0,1040.81446,845.73324,0.0,162.101250,0.0,0.0,0.000000,0.0,0.000000,121.29756,-3.141607,16.604248


In [607]:
new_data["CAISO"]["energy_rt1"] = new_data["CAISO"]["energy_rt"][['INTERVALSTARTTIME_GMT', 'INTERVALENDTIME_GMT', 'OPR_DT',
       'OPR_HR', 'NODE_ID_XML', 'NODE_ID', 'NODE', 'MARKET_RUN_ID', 'LMP_TYPE',
       'XML_DATA_ITEM', 'PNODE_RESMRID', 'GRP_TYPE', 'POS', 'VALUE',
       'OPR_INTERVAL', 'GROUP', 'time_tomerge']][new_data["CAISO"]["energy_rt"]["INTERVALSTARTTIME_GMT"].notna()]
new_data["CAISO"]["energy_rt2"] = new_data["CAISO"]["energy_rt"][['Time', 'Market', 'Location',
       'Location Type', 'LMP', 'Energy', 'Congestion', 'Loss', 'time_tomerge']][new_data["CAISO"]["energy_rt"]["INTERVALSTARTTIME_GMT"].isna()]
new_data["CAISO"]["energy_rt2"]["time_tomerge"] = pd.to_datetime(new_data["CAISO"]["energy_rt2"]["Time"])
print(new_data["CAISO"]["energy_rt1"])

             INTERVALSTARTTIME_GMT        INTERVALENDTIME_GMT      OPR_DT  \
0        2019-11-11T08:00:00-00:00  2019-11-11T08:05:00-00:00  2019-11-11   
1        2019-11-11T08:00:00-00:00  2019-11-11T08:05:00-00:00  2019-11-11   
2        2019-11-11T08:00:00-00:00  2019-11-11T08:05:00-00:00  2019-11-11   
3        2019-11-11T08:00:00-00:00  2019-11-11T08:05:00-00:00  2019-11-11   
4        2019-11-11T08:00:00-00:00  2019-11-11T08:05:00-00:00  2019-11-11   
...                            ...                        ...         ...   
3378199  2022-01-01T07:55:00-00:00  2022-01-01T08:00:00-00:00  2021-12-31   
3378200  2022-01-01T07:55:00-00:00  2022-01-01T08:00:00-00:00  2021-12-31   
3378201  2022-01-01T07:55:00-00:00  2022-01-01T08:00:00-00:00  2021-12-31   
3378202  2022-01-01T07:55:00-00:00  2022-01-01T08:00:00-00:00  2021-12-31   
3378203  2022-01-01T07:55:00-00:00  2022-01-01T08:00:00-00:00  2021-12-31   

         OPR_HR       NODE_ID_XML           NODE_ID              NODE  \
0 

In [609]:
new_data["CAISO"]["total"] = new_data["CAISO"]["total"].merge(add_caiso_energy(new_data["CAISO"]["energy_rt1"], "MCC", "congestion_rt"), on = "time_tomerge",how="left")
new_data["CAISO"]["total"] = new_data["CAISO"]["total"].merge(add_caiso_energy(new_data["CAISO"]["energy_rt1"], "MCE", "energy_rt"), on = "time_tomerge",how="left")
new_data["CAISO"]["total"] = new_data["CAISO"]["total"].merge(add_caiso_energy(new_data["CAISO"]["energy_rt1"], "MCL", "losses_rt"), on = "time_tomerge", how="left")
print(new_data["CAISO"]["total"])

18766
18766
18766
             time_tomerge  ancillary_da_nr  ancillary_da_rd  ancillary_da_rmd  \
0     2019-01-01 00:00:00              NaN              NaN               NaN   
1     2019-01-01 01:00:00          73.6890         4587.144               0.0   
2     2019-01-01 02:00:00          70.5892         1500.000               0.0   
3     2019-01-01 03:00:00          71.0776         1500.000               0.0   
4     2019-01-01 04:00:00          73.1093         1500.000               0.0   
...                   ...              ...              ...               ...   
35060 2022-12-31 20:00:00         101.5209         5700.000               0.0   
35061 2022-12-31 21:00:00          90.5520         6612.000               0.0   
35062 2022-12-31 22:00:00          87.7104         8850.000               0.0   
35063 2022-12-31 23:00:00          84.4800         5670.000               0.0   
35064 2023-01-01 00:00:00          80.8572         7650.000               0.0   

       an

In [610]:
new_data["CAISO"]["total"]["time_tomerge"] = pd.to_datetime(new_data["CAISO"]["total"]["time_tomerge"], utc=True)
new_data["CAISO"]["total"] = new_data["CAISO"]["total"].set_index("time_tomerge")

In [611]:
subset = new_data["CAISO"]["energy_rt2"]
subset["time_tomerge"] = pd.to_datetime(subset["time_tomerge"], utc = True).dt.floor("H")
subset = pd.DataFrame(subset.groupby(["time_tomerge"])["Energy", "Congestion", "Loss"].mean())
subset = subset.drop_duplicates().rename(columns={'Energy':'energy_rt','Congestion':'congestion_rt', "Loss": "losses_rt"})
#print(subset)
new_data["CAISO"]["total"].update(subset)
print(new_data["CAISO"]["total"])

                           ancillary_da_nr  ancillary_da_rd  ancillary_da_rmd  \
time_tomerge                                                                    
2019-01-01 00:00:00+00:00              NaN              NaN               NaN   
2019-01-01 01:00:00+00:00          73.6890         4587.144               0.0   
2019-01-01 02:00:00+00:00          70.5892         1500.000               0.0   
2019-01-01 03:00:00+00:00          71.0776         1500.000               0.0   
2019-01-01 04:00:00+00:00          73.1093         1500.000               0.0   
...                                    ...              ...               ...   
2022-12-31 20:00:00+00:00         101.5209         5700.000               0.0   
2022-12-31 21:00:00+00:00          90.5520         6612.000               0.0   
2022-12-31 22:00:00+00:00          87.7104         8850.000               0.0   
2022-12-31 23:00:00+00:00          84.4800         5670.000               0.0   
2023-01-01 00:00:00+00:00   

/var/folders/cz/nv3pr_214md_31815rhfvdkh0000gn/T/ipykernel_34907/2296779494.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  subset = pd.DataFrame(subset.groupby(["time_tomerge"])["Energy", "Congestion", "Loss"].mean())


In [612]:
load_types = ((new_data["CAISO"]["load"].columns))
#print(np.sum(new_data["CAISO"]["load"]["TAC_AREA_NAME"]==))
print(load_types)

Index(['Unnamed: 0', 'INTERVALSTARTTIME_GMT', 'INTERVALENDTIME_GMT',
       'LOAD_TYPE', 'OPR_DT', 'OPR_HR', 'OPR_INTERVAL', 'MARKET_RUN_ID',
       'TAC_AREA_NAME', 'LABEL', 'XML_DATA_ITEM', 'POS', 'MW',
       'EXECUTION_TYPE', 'GROUP', 'time_tomerge'],
      dtype='object')


In [614]:
def add_caiso_load(sample_data: pd.DataFrame, LOAD_TYPE: str, col_name: str):
    subset = sample_data[(sample_data["EXECUTION_TYPE"] == LOAD_TYPE)]
    subset = subset[subset["OPR_HR"]<25]

    subset = pd.DataFrame(subset.groupby(['OPR_DT', "OPR_HR"])["MW"].mean())
    subset["DATE_HR"] = subset.index.to_numpy()
    subset["time_tomerge"] = subset["DATE_HR"].apply(create_date)
    subset["time_tomerge"] = pd.to_datetime(subset["time_tomerge"], utc = True)

    #subset["time_tomerge"] = subset["time_tomerge"].map(lambda x: x.tz_convert('US/Pacific'))

    subset = subset.drop(columns = ["DATE_HR"])
    subset = subset.reset_index(drop=True)


    subset = subset.rename(columns={"MW": col_name})
    print(len(subset))
    return subset

In [615]:
new_data["CAISO"]["total"] = new_data["CAISO"]["total"].merge(add_caiso_load(new_data["CAISO"]["load"][new_data["CAISO"]["load"]["TAC_AREA_NAME"] == "CA ISO-TAC"], "2DA", "load_2da"), on = "time_tomerge", how="left")
new_data["CAISO"]["total"] = new_data["CAISO"]["total"].merge(add_caiso_load(new_data["CAISO"]["load"][new_data["CAISO"]["load"]["TAC_AREA_NAME"] == "CA ISO-TAC"], "7DA", "load_7da"), on = "time_tomerge", how="left")
new_data["CAISO"]["total"] = new_data["CAISO"]["total"].merge(add_caiso_load(new_data["CAISO"]["load"][new_data["CAISO"]["load"]["TAC_AREA_NAME"] == "CA ISO-TAC"], "ACTUAL", "load_rt"), on = "time_tomerge",how="left")
new_data["CAISO"]["total"] = new_data["CAISO"]["total"].merge(add_caiso_load(new_data["CAISO"]["load"][new_data["CAISO"]["load"]["TAC_AREA_NAME"] == "CA ISO-TAC"], "DAM", "load_da"), on = "time_tomerge",how="left")
new_data["CAISO"]["total"] = new_data["CAISO"]["total"].merge(add_caiso_load(new_data["CAISO"]["load"][new_data["CAISO"]["load"]["TAC_AREA_NAME"] == "CA ISO-TAC"], "RTPD", "load_rtpd"), on = "time_tomerge",how="left")
new_data["CAISO"]["total"] = new_data["CAISO"]["total"].merge(add_caiso_load(new_data["CAISO"]["load"][new_data["CAISO"]["load"]["TAC_AREA_NAME"] == "CA ISO-TAC"], "RTD", "load_rt.1"), on = "time_tomerge",how="left")

print(new_data["CAISO"]["total"])

35012
35012
35012
35012
34990
35006
                   time_tomerge  ancillary_da_nr  ancillary_da_rd  \
0     2019-01-01 00:00:00+00:00              NaN              NaN   
1     2019-01-01 01:00:00+00:00          73.6890         4587.144   
2     2019-01-01 02:00:00+00:00          70.5892         1500.000   
3     2019-01-01 03:00:00+00:00          71.0776         1500.000   
4     2019-01-01 04:00:00+00:00          73.1093         1500.000   
...                         ...              ...              ...   
35060 2022-12-31 20:00:00+00:00         101.5209         5700.000   
35061 2022-12-31 21:00:00+00:00          90.5520         6612.000   
35062 2022-12-31 22:00:00+00:00          87.7104         8850.000   
35063 2022-12-31 23:00:00+00:00          84.4800         5670.000   
35064 2023-01-01 00:00:00+00:00          80.8572         7650.000   

       ancillary_da_rmd  ancillary_da_rmu  ancillary_da_ru  ancillary_da_sr  \
0                   NaN               NaN              N

In [616]:
#new_data["CAISO"]["generation"] = new_data["CAISO"]["generation"].drop(columns=["Unnamed: 0", "Unnamed: 0.1"])
#new_data["CAISO"]["generation"]["time_tomerge"] = pd.to_datetime(new_data["CAISO"]["generation"]["Time"])
new_data["CAISO"]["generation"]["time_tomerge"] = new_data["CAISO"]["generation"]["time_tomerge"] + timedelta(hours=7)
new_data["CAISO"]["generation"]

,Time,Solar,Wind,Geothermal,Biomass,Biogas,Small Hydro,Coal,Nuclear,Natural Gas,Large Hydro,Batteries,Imports,Other,time_tomerge
3,2019-01-01 00:00:00-08:00,0.0,2810.0,993.0,380.0,225.0,200.0,11.0,2273.0,7326.0,1924.0,6.0,6254.0,0.0,2019-01-01 08:00:00+00:00
44,2019-01-01 00:55:00-08:00,0.0,2618.0,992.0,377.0,207.0,192.0,11.0,2274.0,6728.0,1954.0,53.0,6270.0,0.0,2019-01-01 08:00:00+00:00
42,2019-01-01 00:50:00-08:00,0.0,2693.0,992.0,377.0,207.0,193.0,11.0,2273.0,6693.0,1952.0,63.0,6323.0,0.0,2019-01-01 08:00:00+00:00
39,2019-01-01 00:45:00-08:00,0.0,2735.0,992.0,377.0,218.0,197.0,11.0,2273.0,6761.0,1902.0,33.0,6352.0,0.0,2019-01-01 08:00:00+00:00
30,2019-01-01 00:35:00-08:00,0.0,2807.0,993.0,378.0,224.0,203.0,11.0,2273.0,6885.0,1843.0,40.0,6263.0,0.0,2019-01-01 08:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419481,2022-12-31 23:05:00-08:00,-38.0,2994.0,901.0,289.0,211.0,199.0,3.0,2245.0,8705.0,1274.0,420.0,6606.0,0.0,2023-01-01 07:00:00+00:00
419478,2022-12-31 23:00:00-08:00,-37.0,3042.0,901.0,290.0,211.0,199.0,3.0,2245.0,8937.0,1277.0,365.0,6398.0,0.0,2023-01-01 07:00:00+00:00
419507,2022-12-31 23:50:00-08:00,-37.0,2958.0,903.0,286.0,211.0,201.0,3.0,2245.0,8461.0,1353.0,187.0,6441.0,0.0,2023-01-01 07:00:00+00:00
419489,2022-12-31 23:20:00-08:00,-37.0,2837.0,903.0,286.0,212.0,199.0,3.0,2244.0,8505.0,1259.0,605.0,6598.0,0.0,2023-01-01 07:00:00+00:00


In [617]:
new_data["CAISO"]["generation"] = new_data["CAISO"]["generation"].sort_values(by="time_tomerge")
#print(new_data["CAISO"]["generation"])

subset = new_data["CAISO"]["generation"]
subset["time_tomerge"] = pd.to_datetime(subset["time_tomerge"], utc=True).dt.floor("H")
subset = pd.DataFrame(subset.groupby(["time_tomerge"])[subset.columns[:-1]].mean())
subset["time_tomerge"] =subset.index
subset = subset.reset_index(drop=True)
subset = subset.rename(columns={"Solar": "gen_solar", "Wind": "gen_wind", "Geothermal": "gen_geothermal", "Biomass": "gen_biomass", "Biogas": "gen_biogas", "Small Hydro": "gen_small_hydro", "Coal": "gen_coal", "Nuclear": "gen_nuclear", "Natural Gas": "gen_natural_gas", "Large Hydro": "gen_hydro", "Batteries": "gen_batteries", "Imports": "gen_imports", "Other": "gen_other"})
print(subset)
print(len(subset))
new_data["CAISO"]["total"] = new_data["CAISO"]["total"].merge(subset, on = "time_tomerge",how="left")
print(new_data["CAISO"]["total"].set_index("time_tomerge"))
print(len(new_data["CAISO"]["total"]))

       gen_solar     gen_wind  gen_geothermal  gen_biomass  gen_biogas  \
0       0.000000  2812.083333      992.583333   378.500000  220.750000   
1       0.000000  2499.500000      991.833333   378.500000  207.333333   
2       0.000000  2387.083333      992.083333   377.333333  206.166667   
3       0.000000  2369.333333      993.250000   374.750000  205.666667   
4       0.000000  2323.083333      991.583333   377.333333  205.833333   
...          ...          ...             ...          ...         ...   
34983 -27.916667  3714.500000      901.333333   240.750000  214.583333   
34984 -28.500000  4078.750000      900.333333   253.083333  214.333333   
34985 -36.166667  3580.666667      900.916667   286.166667  214.916667   
34986 -37.166667  3161.166667      900.416667   287.416667  211.833333   
34987 -37.166667  2914.750000      902.250000   286.833333  211.250000   

       gen_small_hydro   gen_coal  gen_nuclear  gen_natural_gas    gen_hydro  \
0           200.000000  11.0000

In [618]:
new_data["CAISO"]["total"] = new_data["CAISO"]["total"].set_index("time_tomerge")
print(new_data["CAISO"]["total"])
print(ISO_final["CAISO"])

                           ancillary_da_nr  ancillary_da_rd  ancillary_da_rmd  \
time_tomerge                                                                    
2019-01-01 00:00:00+00:00              NaN              NaN               NaN   
2019-01-01 01:00:00+00:00          73.6890         4587.144               0.0   
2019-01-01 02:00:00+00:00          70.5892         1500.000               0.0   
2019-01-01 03:00:00+00:00          71.0776         1500.000               0.0   
2019-01-01 04:00:00+00:00          73.1093         1500.000               0.0   
...                                    ...              ...               ...   
2022-12-31 20:00:00+00:00         101.5209         5700.000               0.0   
2022-12-31 21:00:00+00:00          90.5520         6612.000               0.0   
2022-12-31 22:00:00+00:00          87.7104         8850.000               0.0   
2022-12-31 23:00:00+00:00          84.4800         5670.000               0.0   
2023-01-01 00:00:00+00:00   

In [621]:
final = pd.concat([ISO_final["CAISO"], new_data["CAISO"]["total"]])
final.to_csv("/Users/evanlipton/Dropbox (YSE)/Solar_ElectricData/Data/ISOs/CAISO.csv")

In [622]:
new_data["CAISO"]["total"].to_csv("/Users/evanlipton/Dropbox (YSE)/Solar_ElectricData/Data/CAISO/all_data.csv")

In [15]:
##CAISO
for key in new_data["CAISO"]:
    print(key)

ancillary_res_rt
ancillary_req_rt
gas_price
energy_da
energy_rt
generation
load
ancillary_req_da
ancillary_res_da


In [35]:
print(new_data["CAISO"]["ancillary_res_da"].iloc[:50,:])

    Unnamed: 0      INTERVALSTARTTIME_GMT        INTERVALENDTIME_GMT  \
0         1424  2019-01-01T08:00:00-00:00  2019-01-01T09:00:00-00:00   
1         1217  2019-01-01T08:00:00-00:00  2019-01-01T09:00:00-00:00   
2         1242  2019-01-01T08:00:00-00:00  2019-01-01T09:00:00-00:00   
3         1724  2019-01-01T08:00:00-00:00  2019-01-01T09:00:00-00:00   
4         1725  2019-01-01T08:00:00-00:00  2019-01-01T09:00:00-00:00   
5         1726  2019-01-01T08:00:00-00:00  2019-01-01T09:00:00-00:00   
6         1727  2019-01-01T08:00:00-00:00  2019-01-01T09:00:00-00:00   
7         1728  2019-01-01T08:00:00-00:00  2019-01-01T09:00:00-00:00   
8         1729  2019-01-01T08:00:00-00:00  2019-01-01T09:00:00-00:00   
9          101  2019-01-01T08:00:00-00:00  2019-01-01T09:00:00-00:00   
10         100  2019-01-01T08:00:00-00:00  2019-01-01T09:00:00-00:00   
11          99  2019-01-01T08:00:00-00:00  2019-01-01T09:00:00-00:00   
12          98  2019-01-01T08:00:00-00:00  2019-01-01T09:00:00-0

In [20]:
print(new_data["CAISO"]["ancillary_res_rt"].iloc[50:100,:])


    Unnamed: 0      INTERVALSTARTTIME_GMT        INTERVALENDTIME_GMT  \
50        6528  2019-01-01T08:00:00-00:00  2019-01-01T08:15:00-00:00   
51        6536  2019-01-01T08:00:00-00:00  2019-01-01T08:15:00-00:00   
52        6534  2019-01-01T08:00:00-00:00  2019-01-01T08:15:00-00:00   
53        3999  2019-01-01T08:00:00-00:00  2019-01-01T08:15:00-00:00   
54        4001  2019-01-01T08:00:00-00:00  2019-01-01T08:15:00-00:00   
55        4017  2019-01-01T08:00:00-00:00  2019-01-01T08:15:00-00:00   
56        4010  2019-01-01T08:00:00-00:00  2019-01-01T08:15:00-00:00   
57        4009  2019-01-01T08:00:00-00:00  2019-01-01T08:15:00-00:00   
58        6540  2019-01-01T08:00:00-00:00  2019-01-01T08:15:00-00:00   
59        4089  2019-01-01T08:00:00-00:00  2019-01-01T08:15:00-00:00   
60        4090  2019-01-01T08:00:00-00:00  2019-01-01T08:15:00-00:00   
61        4091  2019-01-01T08:00:00-00:00  2019-01-01T08:15:00-00:00   
62        8800  2019-01-01T08:00:00-00:00  2019-01-01T08:15:00-0

In [37]:
print(new_data["CAISO"]["ancillary_res_da"][(new_data["CAISO"]["ancillary_res_da"]["ANC_TYPE"] == "NR") & (new_data["CAISO"]["ancillary_res_da"]["ANC_REGION"] == "AS_CAISO_EXP") & (new_data["CAISO"]["ancillary_res_da"]["RESULT_TYPE"] == "AS_COST")])

         Unnamed: 0      INTERVALSTARTTIME_GMT        INTERVALENDTIME_GMT  \
19              365  2019-01-01T08:00:00-00:00  2019-01-01T09:00:00-00:00   
173             359  2019-01-01T09:00:00-00:00  2019-01-01T10:00:00-00:00   
216             303  2019-01-01T10:00:00-00:00  2019-01-01T11:00:00-00:00   
373             297  2019-01-01T11:00:00-00:00  2019-01-01T12:00:00-00:00   
459             291  2019-01-01T12:00:00-00:00  2019-01-01T13:00:00-00:00   
...             ...                        ...                        ...   
3646272       75289  2023-01-01T03:00:00-00:00  2023-01-01T04:00:00-00:00   
3646372       75283  2023-01-01T04:00:00-00:00  2023-01-01T05:00:00-00:00   
3646495       75265  2023-01-01T05:00:00-00:00  2023-01-01T06:00:00-00:00   
3646568       75257  2023-01-01T06:00:00-00:00  2023-01-01T07:00:00-00:00   
3646764       75253  2023-01-01T07:00:00-00:00  2023-01-01T08:00:00-00:00   

             OPR_DT  OPR_HR  OPR_INTERVAL ANC_TYPE    ANC_REGION  \
19     

In [17]:
print(len(new_data["CAISO"]["ancillary_res_rt"]))
print(len(np.unique(new_data["CAISO"]["ancillary_res_rt"]["time_tomerge"])))


14289960
137280


In [168]:
FILE_PATH = "/Users/evanlipton/Dropbox (YSE)/Solar_ElectricData/Data"
new_data = {}
for ISO in ISOS:
    new_data[ISO] = {}#pd.DataFrame({"time_tomerge": pd.date_range("2019-01-01","2022-12-31",freq='h'), "time_tomerge_utc": pd.date_range("2019-01-01","2022-12-31",freq='h', tz="UTC")})
    folder = f"{FILE_PATH}/{ISO}/new/"
    sub_folders = [name for name in os.listdir(folder) if os.path.isdir(os.path.join(folder, name))]

    for folder in sub_folders:
        temp = pd.DataFrame()
        sub_folder = f"{FILE_PATH}/{ISO}/new/{folder}"
        files = [f for f in listdir(sub_folder) if isfile(join(sub_folder, f)) and f[0]!="."]
        for file in tqdm(files):
            #print(f"{sub_folder}/{file}")
            temp = pd.concat([temp, pd.read_csv(f"{sub_folder}/{file}")])
        temp["time_tomerge"] = pd.to_datetime(temp.iloc[:,0])
        temp = temp.sort_values(by="time_tomerge")
        new_data[ISO][sub_folder] = temp

        #try:
        #    new_data[ISO] = new_data[ISO].merge(temp, on=["time_tomerge"])
        #except KeyError:
        #    new_data[ISO] = new_data[ISO].merge(temp, left_on=["time_tomerge_utc"], right_on=["time_tomerge"])
        #print(folder)
        #print(new_data[ISO])

100%|██████████| 47/47 [05:24<00:00,  6.90s/it]
0it [00:00, ?it/s]


IndexError: single positional indexer is out-of-bounds

In [170]:
for key in new_data:
    for k in new_data[key]:
        new_data[key][k].to_csv(f"{k}.csv")

In [187]:
for k in new_data["NYISO"]:
    new_data["NYISO"][k]["time_tomerge"] = pd.to_datetime(new_data["NYISO"][k].iloc[:,0])
    if new_data["NYISO"][k].columns[0]=="Unnamed: 0":
        new_data["NYISO"][k] = new_data["NYISO"][k].drop(columns = ['Unnamed: 0', "time_tomerge"])
        new_data["NYISO"][k]["time_tomerge"] = pd.datetime(new_data["NYISO"][k].iloc[:,0])
    if "time_tomerge" not in new_data["NYISO"][k].columns:
        new_data["NYISO"][k]["time_tomerge"] = pd.to_datetime(new_data["NYISO"][k].iloc[:,0], utc =True)
    print(k)
    print(new_data["NYISO"][k].head())

/Users/evanlipton/Dropbox (YSE)/Solar_ElectricData/Data/NYISO/new/energy_da
                        Time            Market Location Location Type     LMP  \
0  2021-01-01 00:00:00-05:00  DAY_AHEAD_HOURLY   CAPITL          Zone   25.72   
0  2020-01-01 00:00:00-05:00  DAY_AHEAD_HOURLY   CAPITL          Zone   21.77   
0  2019-01-01 00:00:00-05:00  DAY_AHEAD_HOURLY   CAPITL          Zone   26.40   
0  2018-01-01 00:00:00-05:00  DAY_AHEAD_HOURLY   CAPITL          Zone  152.87   
0  2022-01-01 00:00:00-05:00  DAY_AHEAD_HOURLY   CAPITL          Zone   30.95   

   Energy  Congestion  Loss               time_tomerge  
0   12.55      -12.31  0.86  2021-01-01 00:00:00-05:00  
0    6.59      -14.75  0.43  2020-01-01 00:00:00-05:00  
0   10.35      -15.32  0.73  2019-01-01 00:00:00-05:00  
0   48.09     -101.03  3.75  2018-01-01 00:00:00-05:00  
0   26.44       -2.76  1.75  2022-01-01 00:00:00-05:00  
/Users/evanlipton/Dropbox (YSE)/Solar_ElectricData/Data/NYISO/new/ancillary_da
      Eastern Da

In [26]:
## CAISO -- merge the values to keep
ISO = "CAISO"
#ancillary_da, generation_final, load_rtpd, load_da,energy_da, load_rtd, gas_price
to_keep = ["ancillary_da", "generation_final", "load_rtpd", "load_da", "energy_da", "load_rtd", "gas_price", "ancillary_rt", "congestion_da", "losses_da"]
ISO_final[ISO] = ISO_data[ISO][to_keep[0]]
for key in to_keep[1:]:
    ISO_final[ISO] = ISO_final[ISO].merge(ISO_data[ISO][key], on = ["time_tomerge"])
print(ISO_final[ISO])

/usr/local/lib/python3.8/site-packages/pandas/core/frame.py:9190: FutureWarning: Passing 'suffixes' which cause duplicate columns {'date_x'} in the result is deprecated and will raise a MergeError in a future version.
  return merge(


                 date_x  caiso_ancillary_da_nr  caiso_ancillary_da_rd  \
0         1/1/2014 8:00                35.2056                 0.0000   
1         1/1/2014 9:00                36.5024              1111.5000   
2        1/1/2014 10:00                37.5344              1111.5000   
3        1/1/2014 11:00                37.4040              1245.4500   
4        1/1/2014 12:00                37.5640              1389.4500   
...                 ...                    ...                    ...   
46869  12/31/2018 19:00                49.9224              3520.0000   
46870  12/31/2018 20:00                46.9029              4396.0000   
46871  12/31/2018 21:00                40.9629              4530.1095   
46872  12/31/2018 22:00                39.6770               828.7500   
46873  12/31/2018 23:00                32.1937              1414.7840   

       caiso_ancillary_da_rmd  caiso_ancillary_da_rmu  caiso_ancillary_da_ru  \
0                  2840.19220              

In [27]:
#remove the extra date cols
columns_to_keep = [val.lower() for val in list(ISO_final[ISO].columns)]
print(len(columns_to_keep))
columns_to_keep = [val for val in columns_to_keep if val[:5]!="date_"]
print(len(columns_to_keep))
columns_to_keep.remove('unnamed: 0')
columns_to_keep.remove('date')
ISO_final[ISO] = ISO_final[ISO][columns_to_keep]

#set index to the date time
ISO_final[ISO] = ISO_final[ISO].set_index('datetime_tomerge')
print(ISO_final[ISO])

45
37
                           caiso_ancillary_da_nr  caiso_ancillary_da_rd  \
datetime_tomerge                                                          
2014-01-01 08:00:00+00:00                35.2056                 0.0000   
2014-01-01 09:00:00+00:00                36.5024              1111.5000   
2014-01-01 10:00:00+00:00                37.5344              1111.5000   
2014-01-01 11:00:00+00:00                37.4040              1245.4500   
2014-01-01 12:00:00+00:00                37.5640              1389.4500   
...                                          ...                    ...   
2018-12-31 19:00:00+00:00                49.9224              3520.0000   
2018-12-31 20:00:00+00:00                46.9029              4396.0000   
2018-12-31 21:00:00+00:00                40.9629              4530.1095   
2018-12-31 22:00:00+00:00                39.6770               828.7500   
2018-12-31 23:00:00+00:00                32.1937              1414.7840   

                  

In [44]:
#remove iso from column names
for column in ISO_final[ISO].columns:
    if str(ISO).lower() in column:
        print(column.replace(f"{str(ISO).lower()}_",""))
        ISO_final[ISO] = ISO_final[ISO].rename(columns={column: column.replace(f"{str(ISO).lower()}_","")})
print(ISO_final[ISO])

ancillary_da_nr
ancillary_da_rd
ancillary_da_rmd
ancillary_da_rmu
ancillary_da_ru
ancillary_da_sr
load_rtpd
load_2da
load_7da
load_rt_x
load_da
energy_da
load_rt_y
gas_price
ancillary_rt_nr
ancillary_rt_rd
ancillary_rt_rmd
ancillary_rt_rmu
ancillary_rt_ru
ancillary_rt_sr
congestion_da
losses_da
                           ancillary_da_nr  ancillary_da_rd  ancillary_da_rmd  \
datetime_tomerge                                                                
2014-01-01 08:00:00+00:00          35.2056           0.0000        2840.19220   
2014-01-01 09:00:00+00:00          36.5024        1111.5000          14.91310   
2014-01-01 10:00:00+00:00          37.5344        1111.5000           5.70190   
2014-01-01 11:00:00+00:00          37.4040        1245.4500           0.00000   
2014-01-01 12:00:00+00:00          37.5640        1389.4500           0.00000   
...                                    ...              ...               ...   
2018-12-31 19:00:00+00:00          49.9224        3520.0

In [70]:
#NYISO
ISO = "NYISO"
to_keep = ['losses_rt', 'ancillary_rt', 'congestion_rt', 'energy_rt', 'gas_price', 'generation', 'load_forecast', 'energy_da', 'exchange', 'ancillary_da', 'congestion_da', 'losses_da', 'load']

ISO_final[ISO] = ISO_data[ISO][to_keep[0]]
for key in to_keep[1:]:
    ISO_final[ISO] = ISO_final[ISO].merge(ISO_data[ISO][key], on = ["datetime_tomerge"])
print(ISO_final[ISO])

/usr/local/lib/python3.8/site-packages/pandas/core/frame.py:9190: FutureWarning: Passing 'suffixes' which cause duplicate columns {'date_x'} in the result is deprecated and will raise a MergeError in a future version.
  return merge(


                    Date_x  nyiso_losses_rt          datetime_tomerge  \
0      2014-01-01 05:00:00         1.233636 2014-01-01 05:00:00+00:00   
1      2014-01-01 06:00:00         1.190909 2014-01-01 06:00:00+00:00   
2      2014-01-01 07:00:00         1.437273 2014-01-01 07:00:00+00:00   
3      2014-01-01 08:00:00         1.454545 2014-01-01 08:00:00+00:00   
4      2014-01-01 09:00:00         1.519091 2014-01-01 09:00:00+00:00   
...                    ...              ...                       ...   
62405  2018-12-31 19:00:00         1.173636 2018-12-31 19:00:00+00:00   
62406  2018-12-31 20:00:00         1.110909 2018-12-31 20:00:00+00:00   
62407  2018-12-31 21:00:00         1.054545 2018-12-31 21:00:00+00:00   
62408  2018-12-31 22:00:00         0.816364 2018-12-31 22:00:00+00:00   
62409  2018-12-31 23:00:00         0.748182 2018-12-31 23:00:00+00:00   

                 date_x  nyiso_rt_10SR  nyiso_rt_10NSR  nyiso_rt_30OR  \
0         1/1/2014 5:00            0.0            

In [71]:
#remove the extra date cols
columns_to_keep = [val.lower() for val in list(ISO_final[ISO].columns)]
print(len(columns_to_keep))
columns_to_keep = [val for val in columns_to_keep if val[:5] != "date_"]
print(len(columns_to_keep))
columns_to_keep.remove('unnamed: 0')
columns_to_keep.remove('date hour')
columns_to_keep.remove('date')

ISO_final[ISO].columns = ISO_final[ISO].columns.str.lower()
print(ISO_final[ISO].columns)
ISO_final[ISO] = ISO_final[ISO][columns_to_keep]

#set index to the date time
ISO_final[ISO] = ISO_final[ISO].set_index('datetime_tomerge')
print(ISO_final[ISO])

40
30
Index(['date_x', 'nyiso_losses_rt', 'datetime_tomerge', 'date_x',
       'nyiso_rt_10sr', 'nyiso_rt_10nsr', 'nyiso_rt_30or', 'nyiso_rt_regcap',
       'nyiso_rt_regmov', 'date_y', 'nyiso_congestion_rt', 'date',
       'nyiso_energy_rt', 'unnamed: 0', 'nyiso_gas_price', 'date_y',
       'nyiso_gen_dual_fuel', 'nyiso_gen_hydro', 'nyiso_gen_gas',
       'nyiso_gen_nuclear', 'nyiso_gen_other_fossil',
       'nyiso_gen_other_renewables', 'nyiso_gen_wind', 'date_x',
       'nyiso_load_forecast', 'date_y', 'nyiso_energy_da', 'date hour',
       'nyiso_exchange', 'date_x', 'nyiso_da_10sr', 'nyiso_da_10nsr',
       'nyiso_da_30or', 'nyiso_da_regcap', 'date_y', 'nyiso_congestion_da',
       'date_x', 'nyiso_losses_da', 'date_y', 'nyiso_load'],
      dtype='object')
                           nyiso_losses_rt  nyiso_rt_10sr  nyiso_rt_10nsr  \
datetime_tomerge                                                            
2014-01-01 05:00:00+00:00         1.233636            0.0             0.0 

In [72]:
#remove iso from column names
for column in ISO_final[ISO].columns:
    if str(ISO).lower() in column:
        print(column.replace(f"{str(ISO).lower()}_",""))
        ISO_final[ISO] = ISO_final[ISO].rename(columns={column: column.replace(f"{str(ISO).lower()}_","")})
print(ISO_final[ISO])

losses_rt
rt_10sr
rt_10nsr
rt_30or
rt_regcap
rt_regmov
congestion_rt
energy_rt
gas_price
gen_dual_fuel
gen_hydro
gen_gas
gen_nuclear
gen_other_fossil
gen_other_renewables
gen_wind
load_forecast
energy_da
exchange
da_10sr
da_10nsr
da_30or
da_regcap
congestion_da
losses_da
load
                           losses_rt  rt_10sr  rt_10nsr  rt_30or  rt_regcap  \
datetime_tomerge                                                              
2014-01-01 05:00:00+00:00   1.233636      0.0       0.0      0.0   7.000000   
2014-01-01 06:00:00+00:00   1.190909      0.0       0.0      0.0  15.000000   
2014-01-01 07:00:00+00:00   1.437273      0.0       0.0      0.0  15.000000   
2014-01-01 08:00:00+00:00   1.454545      0.0       0.0      0.0  15.000000   
2014-01-01 09:00:00+00:00   1.519091      0.0       0.0      0.0  12.647500   
...                              ...      ...       ...      ...        ...   
2018-12-31 19:00:00+00:00   1.173636      0.0       0.0      0.0  11.582500   
2018-12-31 2

In [88]:
def create_clean_dict_entry(ISO_final: dict, to_keep: list, ISO: str):

    ISO_final[ISO] = ISO_data[ISO][to_keep[0]]
    for key in to_keep[1:]:
        ISO_final[ISO] = ISO_final[ISO].merge(ISO_data[ISO][key], on = ["time_tomerge"])
    print(ISO_final[ISO].columns)
    #remove the extra date cols
    columns_to_keep = [val.lower() for val in list(ISO_final[ISO].columns)]
    print(len(columns_to_keep))
    columns_to_keep = [val for val in columns_to_keep if "date" not in val]
    print(len(columns_to_keep))
    try:
        columns_to_keep.remove('unnamed: 0')
    except:
        pass

    ISO_final[ISO].columns = ISO_final[ISO].columns.str.lower()
    ISO_final[ISO] = ISO_final[ISO][columns_to_keep]

    #set index to the date time
    ISO_final[ISO] = ISO_final[ISO].set_index('time_tomerge')


    #remove iso from column names
    for column in ISO_final[ISO].columns:
        if str(ISO).lower() in column:
            print(column.replace(f"{str(ISO).lower()}_",""))
            ISO_final[ISO] = ISO_final[ISO].rename(columns={column: column.replace(f"{str(ISO).lower()}_","")})
    print(ISO_final[ISO])

In [135]:
for key in ISO_final:
    print(key)
    print(ISO_final[key].shape)

ISONE
(43824, 28)
ERCOT
(43824, 26)
MISO
(43824, 23)
PJM
(43824, 47)
CAISO
(43824, 37)
NYISO
(43824, 28)
SPP
(43824, 21)


In [107]:
to_keep = {}
to_keep["PJM"] = ['ancillary_dasr_procurement', 'ancillary_reg_activation', 'exchange_rt', 'gas_price', 'ancillary_procurement_price', 'energy_da', 'energy_rt', 'load', 'ancillary_nonsynchr_cost', 'generation', 'wind', 'ancillary_procurement_quantity']
to_keep["NYISO"] = ['losses_rt', 'ancillary_rt', 'congestion_rt', 'energy_rt', 'gas_price', 'generation', 'load_forecast', 'energy_da', 'exchange', 'ancillary_da', 'congestion_da', 'losses_da', 'load']
to_keep["CAISO"] = ["ancillary_da", "generation_final", "load_rtpd", "load_da", "energy_da", "load_rtd", "gas_price", "ancillary_rt", "congestion_da", "losses_da"]
to_keep["ISONE"] = ['congestion_da', 'losses_da', 'regulation_price', 'load_rt', 'reserves_quantity', 'wind2', 'energy_da', 'solar2', 'reserves_price', 'gas_price', 'generation', 'exchange_rt', 'energy_rt', 'congestion_rt', 'losses_rt', 'regulation_quantity']
to_keep["SPP"] = ['losses_rt', 'exchange', 'energy_rt', 'load', 'congestion_da', 'gas_price', 'energy_da', 'congestion_rt', 'generation', 'losses_da']
to_keep["MISO"] = ['load_da', 'energy_rt', 'generation_rt', 'congestion_rt', 'losses_rt', 'losses_da', 'exchange_rt', 'energy_da', 'generation_da', 'congestion_da', 'gas_price', 'load_rt']
to_keep["ERCOT"] = ['congestion_da', 'ancillary_price', 'gas_price', 'load', 'energy_da', 'energy_rt', 'generation', 'ancillary_quantity', 'congestion_rt']


In [108]:
#print(ISO_data["PJM"]["final"])
ISO_final = {}
for ISO in ISOS:
    create_clean_dict_entry(ISO_final, to_keep[ISO], ISO)

/usr/local/lib/python3.8/site-packages/pandas/core/frame.py:9190: FutureWarning: Passing 'suffixes' which cause duplicate columns {'date_x'} in the result is deprecated and will raise a MergeError in a future version.
  return merge(


Index(['date_x', 'spp_losses_rt', 'time_tomerge', 'datetime_central',
       'spp_exchange_rt', 'date_y', 'spp_energy_rt', 'date_x', 'spp_load_x',
       'date_y', 'spp_congestion_da', 'Unnamed: 0', 'spp_gas_price', 'date_x',
       'spp_energy_da', 'date_y', 'spp_congestion_rt', 'date_x',
       'spp_gen_coal', 'spp_gen_oil', 'spp_gen_hydro', 'spp_gen_gas',
       'spp_gen_nuclear', 'spp_gen_solar', 'spp_gen_waste', 'spp_gen_wind',
       'spp_gen_wasteheat', 'spp_gen_other', 'spp_load_y', 'date_y',
       'spp_losses_da'],
      dtype='object')
31
22
losses_rt
exchange_rt
energy_rt
load_x
congestion_da
gas_price
energy_da
congestion_rt
gen_coal
gen_oil
gen_hydro
gen_gas
gen_nuclear
gen_solar
gen_waste
gen_wind
gen_wasteheat
gen_other
load_y
losses_da
                           losses_rt  exchange_rt  energy_rt       load_x  \
time_tomerge                                                                
2014-03-01 01:00:00+00:00    0.00000          NaN     6.4200  23048.29650   
2014-0

/usr/local/lib/python3.8/site-packages/pandas/core/frame.py:9190: FutureWarning: Passing 'suffixes' which cause duplicate columns {'date_x'} in the result is deprecated and will raise a MergeError in a future version.
  return merge(


Index(['Date_x', 'nyiso_losses_rt', 'time_tomerge', 'date_x', 'nyiso_rt_10SR',
       'nyiso_rt_10NSR', 'nyiso_rt_30OR', 'nyiso_rt_REGCAP', 'nyiso_rt_REGMOV',
       'Date_y', 'nyiso_congestion_rt', 'Date', 'nyiso_energy_rt',
       'Unnamed: 0', 'nyiso_gas_price', 'date_y', 'nyiso_gen_dual_fuel',
       'nyiso_gen_hydro', 'nyiso_gen_gas', 'nyiso_gen_nuclear',
       'nyiso_gen_other_fossil', 'nyiso_gen_other_renewables',
       'nyiso_gen_wind', 'date_x', 'nyiso_load_forecast', 'date_y',
       'nyiso_energy_da', 'Date Hour', 'nyiso_exchange', 'date_x',
       'nyiso_da_10SR', 'nyiso_da_10NSR', 'nyiso_da_30OR', 'nyiso_da_REGCAP',
       'date_y', 'nyiso_congestion_da', 'date_x', 'nyiso_losses_da', 'date_y',
       'nyiso_load'],
      dtype='object')
40
28
losses_rt
rt_10sr
rt_10nsr
rt_30or
rt_regcap
rt_regmov
congestion_rt
energy_rt
gas_price
gen_dual_fuel
gen_hydro
gen_gas
gen_nuclear
gen_other_fossil
gen_other_renewables
gen_wind
load_forecast
energy_da
exchange
da_10sr
da_10nsr
da

/usr/local/lib/python3.8/site-packages/pandas/core/frame.py:9190: FutureWarning: Passing 'suffixes' which cause duplicate columns {'date_x'} in the result is deprecated and will raise a MergeError in a future version.
  return merge(


...

Index(['date_x', 'miso_load_da', 'Date_x', 'miso_energy_rt', 'date_y',
       'date_hour_x', 'miso_gen_rt_coal', 'miso_gen_rt_gas',
       'miso_gen_rt_hydro', 'miso_gen_rt_nuclear', 'miso_gen_rt_other',
       'miso_gen_rt_wind', 'Date_y', 'miso_congestion_rt', 'Date_x',
       'miso_losses_rt', 'Date_y', 'miso_losses_da', 'date_x', 'hour',
       'miso_exchange_rt', 'Date_x', 'miso_energy_da', 'date_y', 'date_hour_y',
       'miso_gen_da_coal', 'miso_gen_da_gas', 'miso_gen_da_hydro',
       'miso_gen_da_nuclear', 'miso_gen_da_other', 'miso_gen_da_wind',
       'Date_y', 'miso_congestion_da', 'Unnamed: 0', 'miso_gas_price', 'date',
       'miso_load_rt', 'time_tomerge'],
      dtype='object')
38
25
load_da
energy_rt
gen_rt_coal
gen_rt_gas
gen_rt_hydro
gen_rt_nuclear
gen_rt_other
gen_rt_wind
congestion_rt
losses_rt
losses_da
exchange_rt
energy_da
gen_da_coal
gen_da_gas
gen_da_hydro
gen_da_nuclear
gen_da_other
gen_da_wind
congestion_da
gas_price
load_rt
Empty DataFrame
Columns: [load_da

/usr/local/lib/python3.8/site-packages/pandas/core/frame.py:9190: FutureWarning: Passing 'suffixes' which cause duplicate columns {'date_x'} in the result is deprecated and will raise a MergeError in a future version.
  return merge(


Index(['date_x', 'ercot_congestion_da', 'time_tomerge', 'date_y', 'hour',
       'ercot_regdn', 'ercot_regup', 'ercot_rrs', 'ercot_nspin', 'Unnamed: 0',
       'ercot_gas_price', 'date_x', 'ercot_load', 'date_y', 'ercot_energy_da',
       'date_x', 'ercot_energy_rt', 'date_y', 'ercot_gen_biomass',
       'ercot_gen_coal', 'ercot_gen_gas', 'ercot_gen_gascc', 'ercot_gen_hydro',
       'ercot_gen_nuclear', 'ercot_gen_other', 'ercot_gen_solar',
       'ercot_gen_wind', 'date_x', 'ercot_quantity_nsrs',
       'ercot_quantity_regdn', 'ercot_quantity_regup', 'ercot_quantity_rrs',
       'date_y', 'ercot_congestion_rt'],
      dtype='object')
34
26
congestion_da
regdn
regup
rrs
nspin
gas_price
load
energy_da
energy_rt
gen_biomass
gen_coal
gen_gas
gen_gascc
gen_hydro
gen_nuclear
gen_other
gen_solar
gen_wind
quantity_nsrs
quantity_regdn
quantity_regup
quantity_rrs
congestion_rt
                           congestion_da      hour  regdn  regup   rrs  nspin  \
time_tomerge                          

/usr/local/lib/python3.8/site-packages/pandas/core/frame.py:9190: FutureWarning: Passing 'suffixes' which cause duplicate columns {'date_x'} in the result is deprecated and will raise a MergeError in a future version.
  return merge(
/usr/local/lib/python3.8/site-packages/pandas/core/frame.py:9190: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Date_x'} in the result is deprecated and will raise a MergeError in a future version.
  return merge(


Index(['Date_x', 'isone_congestion_da', 'time_tomerge', 'Date_y',
       'isone_losses_da', 'date_x', 'date.1', 'isone_regulation_price',
       'date_y', 'isone_load_rt', 'date_x', 'isone_reserves_quantity',
       'date_y', 'isone_gen_wind2', 'Date_x', 'isone_energy_da', 'date_x',
       'isone_gen_solar2', 'date_y', 'isone_reserves_price', 'Unnamed: 0',
       'isone_gas_price', 'date_x', 'isone_gen_coal', 'isone_gen_hydro',
       'isone_gen_landfillgas', 'isone_gen_gas', 'isone_gen_nuclear',
       'isone_gen_oil', 'isone_gen_other', 'isone_gen_refuse',
       'isone_gen_solar', 'isone_gen_wind', 'isone_gen_wood', 'date_y',
       'isone_exchange_rt', 'Date_y', 'isone_energy_rt', 'Date_x',
       'isone_congestion_rt', 'Date_y', 'isone_losses_rt', 'datetime',
       'isone_regulation_quantity'],
      dtype='object')
44
28
congestion_da
losses_da
regulation_price
load_rt
reserves_quantity
gen_wind2
energy_da
gen_solar2
reserves_price
gas_price
gen_coal
gen_hydro
gen_landfillgas
ge

In [114]:
for key in ISO_final:
    print(key)
    print(len(ISO_final[key]))
    print(ISO_final[key].head())

SPP
42647
                           losses_rt  exchange_rt  energy_rt       load_x  \
time_tomerge                                                                
2014-03-01 01:00:00+00:00      0.000          NaN       6.42  23048.29650   
2014-03-01 02:00:00+00:00     -0.170          NaN      24.53  22885.25983   
2014-03-01 03:00:00+00:00     -0.175          NaN      24.08  22739.41892   
2014-03-01 04:00:00+00:00     -0.250          NaN      24.12  23084.40583   
2014-03-01 05:00:00+00:00     -0.315          NaN      22.40  23693.95733   

                           congestion_da  gas_price  energy_da  congestion_rt  \
time_tomerge                                                                    
2014-03-01 01:00:00+00:00          6.090      6.581      25.80           0.00   
2014-03-01 02:00:00+00:00          5.165      6.581      24.51           0.00   
2014-03-01 03:00:00+00:00          3.860      6.581      23.52           0.00   
2014-03-01 04:00:00+00:00          4.860     

In [113]:
print(ISO_data["ERCOT"]["final"])

                datetime_central        date      hour  ercot_regdn  \
0      2014-01-01 00:00:00-06:00    1/1/2014      1:00         5.65   
1      2014-01-01 01:00:00-06:00    1/1/2014      2:00         5.55   
2      2014-01-01 02:00:00-06:00    1/1/2014      3:00         5.55   
3      2014-01-01 03:00:00-06:00    1/1/2014      4:00         8.11   
4      2014-01-01 04:00:00-06:00    1/1/2014      5:00         8.11   
...                          ...         ...       ...          ...   
43819  2018-12-31 19:00:00-06:00  12/31/2018     20:00         4.01   
43820  2018-12-31 20:00:00-06:00  12/31/2018     21:00         3.50   
43821  2018-12-31 21:00:00-06:00  12/31/2018     22:00         4.01   
43822  2018-12-31 22:00:00-06:00  12/31/2018     23:00         7.15   
43823  2018-12-31 23:00:00-06:00  12/31/2018  24:00:00         4.58   

       ercot_regup  ercot_rrs  ercot_nspin  ercot_quantity_nsrs  \
0             9.72      11.72         1.00               1500.0   
1            

In [51]:
print(ISO_data[ISO]["all_data"])

                datetime_eastern              date  nyiso_da_10SR  \
0      2014-01-01 00:00:00-05:00     1/1/2014 0:00       3.640000   
1      2014-01-01 01:00:00-05:00     1/1/2014 1:00       3.640000   
2      2014-01-01 02:00:00-05:00     1/1/2014 2:00       0.900000   
3      2014-01-01 03:00:00-05:00     1/1/2014 3:00       0.900000   
4      2014-01-01 04:00:00-05:00     1/1/2014 4:00       0.900000   
...                          ...               ...            ...   
43819  2018-12-31 19:00:00-05:00  12/31/2018 19:00       5.976364   
43820  2018-12-31 20:00:00-05:00  12/31/2018 20:00       3.500000   
43821  2018-12-31 21:00:00-05:00  12/31/2018 21:00       3.000000   
43822  2018-12-31 22:00:00-05:00  12/31/2018 22:00       3.000000   
43823  2018-12-31 23:00:00-05:00  12/31/2018 23:00       3.000000   

       nyiso_da_10NSR  nyiso_da_30OR  nyiso_da_REGCAP            date.1  \
0                1.18           0.54             7.00     1/1/2014 0:00   
1                1.18

In [58]:

for val in to_keep:
    print(ISO_data["CAISO"][val]["datetime_tomerge"][0])

2014-01-01 00:00:00+00:00
2014-01-01 00:00:00+00:00
2014-01-01 00:00:00+00:00
2014-01-01 00:00:00+00:00
2014-01-01 00:00:00+00:00
2014-01-01 00:00:00+00:00
2014-01-01 08:00:00+00:00
2014-01-01 00:00:00+00:00
2014-01-01 00:00:00+00:00
2014-01-01 00:00:00+00:00


In [8]:
for k in ISO_data[key]:
    print(ISO_data[key][k])

                      Date  isone_congestion_da          datetime_tomerge
0      2015-01-01 00:00:00                  0.0 2015-01-01 00:00:00+00:00
1      2015-01-01 01:00:00                  0.0 2015-01-01 01:00:00+00:00
2      2015-01-01 02:00:00                  0.0 2015-01-01 02:00:00+00:00
3      2015-01-01 03:00:00                  0.0 2015-01-01 03:00:00+00:00
4      2015-01-01 04:00:00                  0.0 2015-01-01 04:00:00+00:00
...                    ...                  ...                       ...
43819  2014-12-31 19:00:00                  0.0 2014-12-31 19:00:00+00:00
43820  2014-12-31 20:00:00                  0.0 2014-12-31 20:00:00+00:00
43821  2014-12-31 21:00:00                  0.0 2014-12-31 21:00:00+00:00
43822  2014-12-31 22:00:00                  0.0 2014-12-31 22:00:00+00:00
43823  2014-12-31 23:00:00                  0.0 2014-12-31 23:00:00+00:00

[43824 rows x 3 columns]
                      Date  isone_losses_da          datetime_tomerge
0      2015-01-0

In [ ]:
combined_data = {}
for key in ISO_data:
    total_df = ISO_data[key][list(ISO_data[key].keys())[0]]
    for k in tqdm(list(ISO_data[key].keys())[1:]):
        #print((total_df['datetime_tomerge'][0]))
        #print((ISO_data[key][k]['datetime_tomerge'][0]))
        total_df = total_df.merge(ISO_data[key][k], how="outer", on=["datetime_tomerge"])
    combined_data[key] = total_df

 17%|█▋        | 2/12 [00:00<00:00, 15.02it/s]/usr/local/lib/python3.8/site-packages/pandas/core/frame.py:9190: FutureWarning: Passing 'suffixes' which cause duplicate columns {'date_x'} in the result is deprecated and will raise a MergeError in a future version.
  return merge(
 15%|█▌        | 2/13 [00:00<00:02,  4.84it/s]/usr/local/lib/python3.8/site-packages/pandas/core/frame.py:9190: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Date_x'} in the result is deprecated and will raise a MergeError in a future version.
  return merge(
 27%|██▋       | 4/15 [00:00<00:00, 18.17it/s]/usr/local/lib/python3.8/site-packages/pandas/core/frame.py:9190: FutureWarning: Passing 'suffixes' which cause duplicate columns {'datetime_ept_x'} in the result is deprecated and will raise a MergeError in a future version.
  return merge(
 53%|█████▎    | 8/15 [00:01<00:02,  3.20it/s]/usr/local/lib/python3.8/site-packages/pandas/core/frame.py:9190: FutureWarning: Passing 'suffixes' which 

In [ ]:
combined_data

In [ ]:
for i in range(len(ISO_data[key][k])):
    print(pd.to_datetime(ISO_data[key][k].iloc[i,0]))



In [130]:
ISO_data[key][k] = ISO_data[key][k][ISO_data[key][k].index % 24 != 0]
print(ISO_data[key][k].iloc[:30,:])

              date    date_hour  miso_gen_da_coal  miso_gen_da_gas  \
1    1/1/2014 1:00   1/1/2014 2           48729.7           8064.0   
2    1/1/2014 2:00   1/1/2014 3           47168.2           8001.2   
3    1/1/2014 3:00   1/1/2014 4           46703.2           7925.6   
4    1/1/2014 4:00   1/1/2014 5           46497.8           7969.3   
5    1/1/2014 5:00   1/1/2014 6           47058.5           8047.6   
6    1/1/2014 6:00   1/1/2014 7           49225.5           8612.5   
7    1/1/2014 7:00   1/1/2014 8           50974.3           9006.8   
8    1/1/2014 8:00   1/1/2014 9           52153.2           9330.7   
9    1/1/2014 9:00  1/1/2014 10           52515.7           9420.0   
10  1/1/2014 10:00  1/1/2014 11           53059.3           9205.0   
11  1/1/2014 11:00  1/1/2014 12           52882.3           8798.4   
12  1/1/2014 12:00  1/1/2014 13           52581.7           8533.6   
13  1/1/2014 13:00  1/1/2014 14           51712.4           8289.6   
14  1/1/2014 14:00  

In [107]:
for name in titles["SPP"]:
    print(ISO_data["SPP"][name])

                      date  spp_losses_rt
0      2014-03-01 01:00:00        0.00000
1      2014-03-01 02:00:00       -0.17000
2      2014-03-01 03:00:00       -0.17500
3      2014-03-01 04:00:00       -0.25000
4      2014-03-01 05:00:00       -0.31500
...                    ...            ...
42402  2018-12-31 19:00:00       -0.13040
42403  2018-12-31 20:00:00       -0.14715
42404  2018-12-31 21:00:00       -0.15080
42405  2018-12-31 22:00:00       -0.36170
42406  2018-12-31 23:00:00       -0.09535

[42407 rows x 2 columns]
                datetime_central  spp_exchange_rt
0      2014-01-01 00:00:00-06:00              NaN
1      2014-01-01 01:00:00-06:00              NaN
2      2014-01-01 02:00:00-06:00              NaN
3      2014-01-01 03:00:00-06:00              NaN
4      2014-01-01 04:00:00-06:00              NaN
...                          ...              ...
43819  2018-12-31 19:00:00-06:00              NaN
43820  2018-12-31 20:00:00-06:00              NaN
43821  2018-12-31 21

In [91]:
print(ISO_data['SPP']["exchange"].dropna())

                datetime_central  spp_exchange_rt
1416   2014-03-01 00:00:00-06:00       525.712333
1417   2014-03-01 01:00:00-06:00       488.295500
1418   2014-03-01 02:00:00-06:00       576.432000
1419   2014-03-01 03:00:00-06:00       550.466500
1420   2014-03-01 04:00:00-06:00       516.769833
...                          ...              ...
43795  2018-12-30 19:00:00-06:00      1156.691667
43796  2018-12-30 20:00:00-06:00      1000.100667
43797  2018-12-30 21:00:00-06:00      1091.302333
43798  2018-12-30 22:00:00-06:00      1189.362333
43799  2018-12-30 23:00:00-06:00      1077.332333

[41833 rows x 2 columns]


In [62]:
#working on energy_da

#pjm is the only one with extra columns
ISO_data["PJM"]['energy_da'] = ISO_data["PJM"]['energy_da'][["datetime_ept", "pjm_energy_da"]]

In [63]:

#rename all the columns consistently
for ISO in ISOS:
    ISO_data[ISO]['energy_da'].rename(columns={ISO_data[ISO]['energy_da'].columns[0]: 'date', ISO_data[ISO]['energy_da'].columns[1]: 'energy_da'},inplace=True)
    ISO_data[ISO]['energy_da']['date'] =  pd.to_datetime(ISO_data[ISO]['energy_da']['date'])
    ISO_data[ISO]['energy_da'] = ISO_data[ISO]['energy_da'].sort_values(by="date").reset_index(drop=True)
    #print(ISO_data[ISO]['energy_da'])

In [10]:
ISO_data["SPP"]['energy_da']

,date,energy_da
0,2014-03-01 01:00:00,25.8000
1,2014-03-01 02:00:00,24.5100
2,2014-03-01 03:00:00,23.5200
3,2014-03-01 04:00:00,24.7200
4,2014-03-01 05:00:00,24.0600
...,...,...
42402,2018-12-31 19:00:00,17.4804
42403,2018-12-31 20:00:00,16.7689
42404,2018-12-31 21:00:00,15.4343
42405,2018-12-31 22:00:00,15.7768


In [64]:
def add_new_data(ISO_data: dict, ISO: str, product: str, path: str, years: list, col_0: str, col_1: str):
    temp = pd.DataFrame()
    for year in years:
        temp = pd.concat([temp, pd.read_csv(f"{path}{year}.csv")])
    temp = temp[[col_0, col_1]]
    temp.rename(columns={temp.columns[0]: 'date', temp.columns[1]: 'energy_da'},inplace=True)
    temp['date'] = pd.to_datetime(temp['date'])
    temp = temp.drop_duplicates(subset =["date"])
    print(np.setdiff1d(temp.columns,ISO_data[ISO][product].columns))
    if len(np.setdiff1d(temp.columns, ISO_data[ISO][product].columns)) == 0:
        print("here")
        ISO_data[ISO][product] = pd.concat([ISO_data[ISO][product], temp]).reset_index(drop=True)
    print(ISO_data[ISO][product].head())
    print(ISO_data[ISO][product].tail())


In [85]:
import gridstatus
iso = gridstatus.MISO()

df = iso.get_lmp(date = "latest",market= "DAY_AHEAD_HOURLY")
print(df)

TypeError: Cannot localize tz-aware Timestamp, use tz_convert for conversions

In [66]:
years = ["2019", "2020", "2021", "2022"]
path = "/Users/evanlipton/Dropbox (YSE)/Solar_ElectricData/Data/NYISO/new/get_lmp/DAY_AHEAD_HOURLY/NYISO_get_lmp_"
ISO = "NYISO"
product = "energy_da"
col_0 = "Time"
col_1 = "Energy"
add_new_data(ISO_data, ISO, product, path, years, col_0,col_1)


[]
here
                  date  energy_da
0  2014-01-01 00:00:00      55.31
1  2014-01-01 01:00:00      40.51
2  2014-01-01 02:00:00      55.48
3  2014-01-01 03:00:00      42.26
4  2014-01-01 04:00:00      42.22
                            date  energy_da
78883  2022-12-31 19:00:00-05:00      32.48
78884  2022-12-31 20:00:00-05:00      32.46
78885  2022-12-31 21:00:00-05:00      30.45
78886  2022-12-31 22:00:00-05:00      28.33
78887  2022-12-31 23:00:00-05:00      21.41


In [52]:
ISO_data[ISO][product]

,date,energy_da
0,2014-01-01 00:00:00,35.70
1,2014-01-01 01:00:00,31.80
2,2014-01-01 02:00:00,31.70
3,2014-01-01 03:00:00,31.70
4,2014-01-01 04:00:00,31.61
...,...,...
200836,2022-12-30 20:00:00,40.88
200859,2022-12-30 21:00:00,37.00
200882,2022-12-30 22:00:00,37.40
200905,2022-12-30 23:00:00,33.78


In [95]:
import io
import dropbox
token = "sl.BZLpDVdDsvSMDB6KOtUkcE0LeS6ngGQGn-FZO9D6q4Cv_BFY870wRmNEI1BGzraxlewIl_Fq98vOZbM02GSIfyNmshfYBK1qbHvOvWuUa5BCnNedn3Uf7fQWw1MyEbWnA-VOj2IM"
DBX = dropbox.Dropbox(token)

_, res = DBX.files_download("/Users/evanlipton/Dropbox (YSE)/Solar_ElectricData/Data/PJM/new/raw data/da_hrl_lmps_2019.csv")

with io.BytesIO(res.content) as stream:
    df = pd.read_csv(stream, index_col=0)

ApiError: ApiError('99b7049b9ed8456aa3e0c1f1a1220702', DownloadError('path', LookupError('not_found', None)))

In [110]:

print((ISO_data["PJM"]["energy_da"]))
years = ["2019", "2020", "2021", "2022"]
path = "/Users/evanlipton/Dropbox (YSE)/Solar_ElectricData/Data/PJM/new/raw data/da_hrl_lmps_"
temp =
for year in years:
    temp = pd.concat([temp, pd.read(f"{path}{year}")])


SyntaxError: invalid syntax (2697313729.py, line 4)

In [60]:
for column in ISO_data["NYISO"]['energy_da'].columns:
    print(type(ISO_data["NYISO"]['energy_da'][column][0]))

<class 'str'>
<class 'numpy.float64'>


In [64]:
ISO_data["PJM"]['energy_da']ISO_data["PJM"]['energy_da'][['datetime_ept', "pjm_energy_da"]]

,date,energy_da,pjm_energy_da,pjm_lmp_da,pjm_congestion_da,pjm_losses_da
0,2014-01-01 05:00:00,1/1/2014 0:00,35.70,35.827291,0.092679,0.034612
1,2014-01-01 06:00:00,1/1/2014 1:00,31.80,31.938325,0.086797,0.051528
2,2014-01-01 07:00:00,1/1/2014 2:00,31.70,31.830537,0.080422,0.050115
3,2014-01-01 08:00:00,1/1/2014 3:00,31.70,31.816503,0.064057,0.052446
4,2014-01-01 09:00:00,1/1/2014 4:00,31.61,31.724109,0.061574,0.052535
...,...,...,...,...,...,...
43819,2019-01-01 00:00:00,12/31/2018 19:00,24.27,24.551853,0.144884,0.136969
43820,2019-01-01 01:00:00,12/31/2018 20:00,23.14,23.408255,0.148338,0.119918
43821,2019-01-01 02:00:00,12/31/2018 21:00,22.27,22.550193,0.154141,0.126051
43822,2019-01-01 03:00:00,12/31/2018 22:00,20.55,20.648788,-0.026945,0.125733


In [38]:
url = 'https://data.gridstatus.io/caiso/load/today.csv'
headers = {'x-api-key': '2Vis4Yt4Q149vSQsrh1GJ6lZeI4flFHg9mYDrEGe'}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    df = pd.read_csv(StringIO(response.text))
else:
    print("Error: ", response.status_code)

Error:  429


In [ ]:
dataset_dict = {}
for ISO in ISOS:
    dataset_dict[ISO]["LMP"] =  ["lmp_day_ahead_hourly_zone" "lmp_real_time_5_min_zone"]
    dataset_dict[]["LMP"] = ["lmp_day_ahead_hourly_zone" "lmp_real_time_5_min_zone"]

In [18]:
for i in range(30)
    url = "https://data.gridstatus.io/nyiso/{pricing-dataset}/archive/{date}.csv"

,Time,Load
0,2023-02-18 00:00:00-08:00,22362.0
1,2023-02-18 00:05:00-08:00,22505.0
2,2023-02-18 00:10:00-08:00,22607.0
3,2023-02-18 00:15:00-08:00,22604.0
4,2023-02-18 00:20:00-08:00,22592.0
...,...,...
148,2023-02-18 12:20:00-08:00,16137.0
149,2023-02-18 12:25:00-08:00,16085.0
150,2023-02-18 12:30:00-08:00,16064.0
151,2023-02-18 12:35:00-08:00,16051.0


In [48]:
from datetime import date, timedelta

def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

start_date = date(2018, 12, 1)
end_date = date(2018, 12, 31)
df = pd.DataFrame()
for single_date in tqdm(daterange(start_date, end_date)):
    date = single_date.strftime("%Y-%m-%d").replace("-","")
    print(date)
    url = f"https://data.gridstatus.io/nyiso/lmp_day_ahead_hourly_zone/archive/{date}.csv"
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        print("here")
        df = pd.concat([df,pd.read_csv(StringIO(response.text))])

0it [00:00, ?it/s]

20181201


2it [00:00,  5.03it/s]

20181202
20181203


4it [00:00,  6.61it/s]

...

20181226
20181227


28it [00:03,  8.55it/s]

20181228
20181229


30it [00:03,  7.96it/s]

20181230


In [59]:
import gridstatus
iso = gridstatus.PJM()

In [60]:
df =iso.get_lmp(start = "20191201", end = "20191231", market = "DAY_AHEAD_HOURLY")

/Users/evanlipton/PycharmProjects/gridstatus_forked/gridstatus/pjm.py:322: UserWarning: Querying before archive date, so filtering by location will happen after all data is downloaded
  warnings.warn(
100%|██████████| 92/92 [18:21<00:00, 12.11s/it]


In [61]:
df

,Time,Market,Location,Location Name,Location Type,LMP,Energy,Congestion,Loss
0,2019-12-01 00:00:00-05:00,DAY_AHEAD_HOURLY,51217,EASTERN HUB,HUB,19.348276,18.63,0.162593,0.555683
1,2019-12-01 00:00:00-05:00,DAY_AHEAD_HOURLY,51287,WEST INT HUB,HUB,18.660000,18.63,0.163333,-0.133333
2,2019-12-01 00:00:00-05:00,DAY_AHEAD_HOURLY,51288,WESTERN HUB,HUB,18.904062,18.63,0.160417,0.113645
3,2019-12-01 00:00:00-05:00,DAY_AHEAD_HOURLY,4669664,NEW JERSEY HUB,HUB,18.945231,18.63,0.181373,0.133858
4,2019-12-01 00:00:00-05:00,DAY_AHEAD_HOURLY,33092311,CHICAGO GEN HUB,HUB,17.081231,18.63,-0.565923,-0.982846
...,...,...,...,...,...,...,...,...,...
8635,2019-12-30 23:00:00-05:00,DAY_AHEAD_HOURLY,34497125,AEP GEN HUB,HUB,15.449211,15.73,0.007368,-0.288157
8636,2019-12-30 23:00:00-05:00,DAY_AHEAD_HOURLY,34497127,AEP-DAYTON HUB,HUB,15.680587,15.73,0.004638,-0.054051
8637,2019-12-30 23:00:00-05:00,DAY_AHEAD_HOURLY,34497151,OHIO HUB,HUB,15.672360,15.73,0.002772,-0.060412
8638,2019-12-30 23:00:00-05:00,DAY_AHEAD_HOURLY,35010337,DOMINION HUB,HUB,15.642915,15.73,0.010000,-0.097085
